In [1]:
import os

import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Loss
from scipy import ndimage
from sklearn.model_selection import train_test_split

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
st_idx = 0
ed_idx = 1000

train_data = np.zeros((ed_idx,300,300,3), dtype=np.float32)
for i in range(st_idx, st_idx + ed_idx):
    img = keras.preprocessing.image.load_img('Datasets/crop_data/' + str(i) + '.png')
    img_arr = keras.preprocessing.image.img_to_array(img)
    
    train_data[i - st_idx] = img_arr/255
    
data = pd.read_csv('Datasets/labels.csv')
pair1 = data[['m11','m22']][st_idx:st_idx + ed_idx].to_numpy()
pair2 = data[['m12','m21']][st_idx:st_idx + ed_idx].to_numpy()
pair3 = data[['m31','m32']][st_idx:st_idx + ed_idx].to_numpy()

pair1 = pair1.astype(np.float32)
pair2 = pair2.astype(np.float32)
pair3 = pair3.astype(np.float32)

In [3]:
pair1_model = keras.models.load_model('pair1')
pair2_model = keras.models.load_model('pair2')
pair3_model = keras.models.load_model('pair3')

In [4]:
output1 = pair1_model.predict(train_data)
output2 = pair2_model.predict(train_data)
output3 = pair3_model.predict(train_data)

In [5]:
f = open('train_result.txt', 'w')

for i in range(0,50):
    temp = str(output1[i][0]) + ',' + str(output2[i][0]) + ',' + str(output2[i][1]) + ',' + str(output1[i][1]) + ',' + str(output3[i][0]) + ',' + str(output3[i][1]) + '\n'
    f.write(temp)
f.close()

In [3]:
img2h = keras.models.load_model('img2h_epoch20', compile=False)

In [5]:
outputs = img2h.predict(train_data[:50])

In [12]:
# outputs = img2h.predict(train_data)

f = open('train_result_epoch20_1000.txt', 'w')

for i in range(0,50):
    temp = str(outputs[0][i][0]) + ',' + str(outputs[1][i][0]) + ',' + str(outputs[1][i][1]) + ',' + str(outputs[0][i][1]) + ',' + str(outputs[2][i][0]) + ',' + str(outputs[2][i][1]) + '\n'
    f.write(temp)
f.close()

In [4]:
for i in range(0,1000,50):
    outputs = img2h.predict(train_data[i:i+50])
    
    f = open('train_result_epoch20_1000.txt', 'a')

    for i in range(0,50):
        temp = str(outputs[0][i][0]) + ',' + str(outputs[1][i][0]) + ',' + str(outputs[1][i][1]) + ',' + str(outputs[0][i][1]) + ',' + str(outputs[2][i][0]) + ',' + str(outputs[2][i][1]) + '\n'
        f.write(temp)
    f.close()

In [6]:
data_size = 500
val_data = np.zeros((100,300,300,3), dtype=np.float32)

for i in range(data_size, data_size+100):
    img = keras.preprocessing.image.load_img('Datasets/crop_data/' + str(i) + '.png')
    img_arr = keras.preprocessing.image.img_to_array(img)
    
    val_data[i - data_size] = img_arr/255

v_data = pd.read_csv('Datasets/labels.csv')
v_pair1 = v_data[['m11','m22']][data_size:data_size+100].to_numpy()
v_pair2 = v_data[['m12','m21']][data_size:data_size+100].to_numpy()
v_pair3 = v_data[['m31','m32']][data_size:data_size+100].to_numpy()

v_pair1 = v_pair1.astype(np.float32)
v_pair2 = v_pair2.astype(np.float32)
v_pair3 = v_pair3.astype(np.float32)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss=keras.losses.MeanSquaredError(),
              loss_weights={'idx1':1.0, 'idx2':10.0, 'idx3':1.0 * pow(10,6)})
img2h.evaluate(val_data, {'idx1':v_pair1, 'idx2':v_pair2, 'idx3':v_pair3})

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.